In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy  as np
import pandas as pd

import os

DETDIR='results/Main'
ANNDIR='Annotations'
TRIDIR='ImageSets/Main/'

In [ ]:
detections = os.listdir(DETDIR)
print detections

In [ ]:
tests = None
with open(os.path.join(TRIDIR,'test_4.txt')) as f:
    tests = f.read()
tests = tests.split("\n")

In [ ]:
frames = []
for det in detections:
    frames.append(pd.read_csv(os.path.join(DETDIR,det),
                    delimiter=' ',names=['file','prob','xmin','ymin','xmax','ymax','pred_class']) )

In [ ]:
pred_df = pd.concat(frames,ignore_index=True)

In [ ]:
#Check a single result
pred_df[pred_df.file == 'muminus008457'].sort('prob',ascending=False)

In [ ]:
g = {}
for ix,image in enumerate(tests):
    data = None
    with open(os.path.join(ANNDIR,image+".txt")) as f:
        data = f.read()
    data = data.split(" ")[2:]
    data = np.array(data,dtype=np.float64)
    g[image] = data

In [ ]:
def iou(bbox1,bbox2):
    ixmin = np.maximum(bbox1[0], bbox2[0])
    iymin = np.maximum(bbox1[1], bbox2[1])
    ixmax = np.minimum(bbox1[2], bbox2[2])
    iymax = np.minimum(bbox1[3], bbox2[3])

    iw = np.maximum(ixmax - ixmin, 0.)
    ih = np.maximum(iymax - iymin, 0.)

    inters = iw*ih
             # union                                                                                                        
    uni = ((bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1]) +
           (bbox2[2] - bbox2[0]) *
           (bbox2[3] - bbox2[1]) - inters)
             # IoU                                                                                                          
    iou = inters / uni
    return iou

In [ ]:
def iou_row(row):
    pred_bbox = np.array(row[['xmin','ymin','xmax','ymax']])
    return iou(pred_bbox,g[row.file])

In [ ]:
# Go row by row and calculate iou, very slowly
pred_df['iou'] = pred_df.apply(iou_row,axis=1)

In [ ]:
import re
truth = re.compile("^([a-z]+)[0-9]+")
pars = ['','eminus','proton','pizero','muminus']

def get_gt(row): 
    return pars.index(truth.search(row.file).group(1))
    

In [ ]:
# Go row by row and mark the groundtruth
pred_df['gt'] = pred_df.apply(get_gt,axis=1)

In [ ]:
#Check a single result again
pred_df[pred_df.file == 'eminus019622'].sort('prob',ascending=False)

In [ ]:
ious = []
probs = []
nimages = 0
im = {}
for image in g:
    df = pred_df[pred_df.file == image].sort('prob',ascending=False)
    top = df.iloc[0] # take only the top prediction
    
    im[image] = {'gt':top['gt'],'top_pred_class':top['pred_class'],'top_iou':top['iou'],'top_prob':top['prob']}


In [ ]:
im_df = pd.DataFrame(im).T

In [ ]:
true_particle = [im_df[im_df['gt']==i] for i in xrange(len(pars)) if i != 0]

In [ ]:
len(true_particle)

In [ ]:
matplotlib.rcParams['font.size'] = 16

pthresh = 0.5
for ix, tpar in enumerate(true_particle):
    ii  = ix
    ii += 1
    
    fig, ax = plt.subplots(figsize=(10,6))
    divs = 0.01
    ntot = float(tpar.index.values.size)
    for i in xrange(4):
        i+=1
        s = tpar[ tpar['top_pred_class'] == i ][ tpar['top_prob'] >= pthresh]
        nn = s.top_iou.values.size
        ax.hist(s.top_iou.values,
                bins=np.arange(0,1.0+divs,divs),
                label='Detected: {}\nFrac: {:.2f}'.format(pars[i],
                                                          nn/ntot),
                alpha=0.5)
        
    ax.set_title("Truth: {}".format(pars[ii]),fontweight='bold')
    ax.set_xlabel("IoU @ Prob Thresh: {}".format(pthresh),fontweight='bold')
    plt.legend(loc='best',fontsize=14)
    plt.tight_layout()
    plt.savefig('iou_true_{}.pdf'.format(pars[ii]), format='pdf', dpi=1000)
    plt.show()

In [ ]:
matplotlib.rcParams['font.size'] = 16
fig, _ = plt.subplots(figsize=(10,3*6))
ax = plt.subplot(3,1,1)
ax.hist(ious,bins=np.arange(0,1.0+divs,divs))
ax.set_title("Truth: proton\nDetect: muminus")
ax.set_xlabel('IoU')
ax = plt.subplot(3,1,2)
ax.hist(probs,bins=np.arange(0,1.0+divs,divs))
ax.set_title("Truth: proton\nDetect: muminus")
ax.set_xlabel('Detection Probability')
ax = plt.subplot(3,1,3)
divs = 0.01
ax.hist2d(ious,probs,bins=(np.arange(0,1.0+divs,divs),
                           np.arange(0,1.0+divs,divs)))
plt.tight_layout()
plt.show()